## Tarea VaR

In [59]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from scipy.stats import norm

Importamos nuestros datos y calculamos los rendimientos diarios de cada accion

In [3]:
data=pd.read_excel("PreciosPortf.xlsx",index_col=0)

In [4]:
rend=data.pct_change()
rend

,ASURB MF Equity,GFNORTEO MF Equity,PE&OLES* MF Equity,WALMEX* MF Equity,IENOVA* MF Equity
DATE,,,,,
2016-01-01,NaN,NaN,NaN,NaN,NaN
2016-01-04,-0.022961,-0.033688,-0.007448,-0.014946,0.008847
2016-01-05,-0.022074,-0.000763,0.003127,-0.004202,-0.020554
2016-01-06,-0.004549,-0.000981,-0.021366,-0.006564,-0.007555
2016-01-07,-0.005432,-0.022809,0.001506,-0.056395,0.019312
2016-01-08,-0.004118,-0.007706,-0.003180,0.026007,-0.017148
2016-01-11,-0.008400,0.005065,-0.000638,0.041433,0.009146
2016-01-12,0.017469,0.000448,-0.051602,0.004447,0.007250
2016-01-13,-0.022001,-0.008059,-0.011323,0.007223,-0.021595


In [131]:
#Calculamos la esperanza anual del rendimiento de cada Accion
rend_ports=rend.mean()*252
rend_ports

ASURB MF Equity       0.123539
GFNORTEO MF Equity    0.078811
PE&OLES* MF Equity    0.111053
WALMEX* MF Equity     0.082426
IENOVA* MF Equity     0.078997
dtype: float64

In [132]:
#Matriz de Varianza-Covarianza
VarCov= rend.cov()
VarCov

,ASURB MF Equity,GFNORTEO MF Equity,PE&OLES* MF Equity,WALMEX* MF Equity,IENOVA* MF Equity
ASURB MF Equity,0.000224,0.000086,0.000061,0.000036,0.000044
GFNORTEO MF Equity,0.000086,0.000341,0.000094,0.000082,0.000082
PE&OLES* MF Equity,0.000061,0.000094,0.000548,0.000041,0.000045
WALMEX* MF Equity,0.000036,0.000082,0.000041,0.000215,0.000043
IENOVA* MF Equity,0.000044,0.000082,0.000045,0.000043,0.000226


##### Definimos las funciones port_rend y port_vol para calcular la esperanza y desviacion estandar de un portafolio

In [133]:
def port_rend(weights, r):
    E_p = (r.mean() @ weights) * 252
    return E_p

def port_vol(weights, r):
    S_p = np.sqrt(weights.T @ (r.cov() * 252) @ weights)
    return S_p

##### Encontramos los pesos que minimicen el ratio de Sharpe

In [155]:
N, M = rend.shape
w0  = np.random.randn(M)
rf = 0.06

def sum_weights(weights):
    return weights.sum() - 1 # == 0

def min_func_sharpe(weights):
    return - (port_rend(weights, rend) - rf) / port_vol(weights, rend)

constraints = [
    {"type": "eq", "fun": sum_weights}
]

f_sharpe = minimize(min_func_sharpe, w0, constraints=constraints)

In [157]:
w_sharpe = f_sharpe.x

In [159]:
w_sharpe

array([ 0.73128983, -0.140285  ,  0.17596234,  0.16472535,  0.06830749])

Utilizando los pesos optimizados con base en Sharpe encontramos la Varianza y Desviacion Estandar Diaria y Anual para el portafolio

In [162]:
varianza_diaria=np.dot(w_opt,np.dot(VarCov,w_opt.T))
desvstd_diaria=np.sqrt(varianza_diaria)

In [163]:
desvstd_anual=desvstd_diaria*np.sqrt(252)
varianza_anual=desvstd_anual**2

De la matriz de Var-Cov extraemos la varianza de cada accion individual (Se utilizara para encontrar nivel de diversificacion)

In [164]:
var_indiv=[]
for i in range(VarCov.shape[0]):
    var_indiv.append(VarCov.values[i,i])
    

##### Definimos la funcion VaR, la cual nos regresara VaR y Diversificacion Anual y Diaria para un portafolio

Para utilizar funcion VaR necesitamos definir el monto a invertir, el nivel de confianza, desviacion estandar diaria y anual, pesos, matriz de Var_Cov y la varianza individual de cada accion

In [165]:
MontoInvertido=10000000
Nivel_Confianza=.95
def VaR(MontoInvertido,Nivel_Confianza,desvstd_diaria,desvstd_anual,w_sharpe,VarCov,var_indiv):
    SN=norm.ppf(Nivel_Confianza)
    VarAnn= MontoInvertido*SN*desvstd_anual
    VarDaily= MontoInvertido*SN*desvstd_diaria
    Diversificacion_Diario=np.sum(w_opt*MontoInvertido*np.sqrt(var_indiv)*SN)-VarDaily
    Diversificacion_Anual=np.sum(w_opt*MontoInvertido*np.sqrt(var_indiv)*np.sqrt(252)*SN)-VarAnn
    return(f"El VaR y Efecto Diversificacion Anual es: {VarAnn} , {Diversificacion_Anual} El VaR y Efecto Diversificacion Diario es: {VarDaily} , {Diversificacion_Diario}")
    


In [166]:
VaR(MontoInvertido,Nivel_Confianza,desvstd_diaria,desvstd_anual,w_opt,VarCov,var_indiv)

'El VaR y Efecto Diversificacion Anual es: 2552187.383311775 , 1572593.1946311719 El VaR y Efecto Diversificacion Diario es: 160772.69322570562 , 99064.05967777802'